# RagTopia Group - Dataset \#3


*   **Shehi** Adem
*   **Alibeaj** Michael
*   **Barda** Luca
*   **Znazen** Ghassen
*   **Taktak** Youssef

Link to video (https://drive.google.com/drive/folders/1OGedW7IKKdAc55oAAl6swmcDYfpJy9Tb?usp=drive_link)



# 0) Prepare environment

## ⚙️ Settings

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import random
import torch

# Make experiment repeatable
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
torch.random.manual_seed(SEED)

# Configure plot display settings
sns.set(font_scale=1.4)
sns.set_style('white')
plt.rc('font', size=14)

## Load RAG 12000 dataset
The dataset constitutes a triple featured collection, with each entry comprising three distinct fields: context, question, and answer. It is specifically designed to support the development of models optimized for Retrieval-Augmented Generation (RAG). The dataset includes 12,000 entries, all in English.

Each data instance consists of a context passage, a question formulated based on the context, and a corresponding answer. The context is sourced from the Falcon RefinedWeb corpus, while the question and answer pairs are generated using the GPT-4 language model.
The dataset is already partitioned in training and test set with a 80-20 split ratio.

In [ ]:
!pip install -U datasets

In [ ]:
from datasets import load_dataset, concatenate_datasets

# Load the RAG dataset from Hugging Face
dataset = load_dataset("neural-bridge/rag-dataset-12000")

train_set = dataset["train"]  # Train split
test_set = dataset["test"]  # Test split

## Clean the data
Remove examples where any field is empty or just whitespace from both train set and test set. As a result 2 samples are removed from the train set and 1 sample is removed from the test set.

In [ ]:
# Remove examples where any field is empty or just whitespace from both train set and test set
print(f"Original size train: {len(train_set)}")
train_set = train_set.filter(lambda example: bool(example["context"] is not None and example["context"].strip()))
train_set = train_set.filter(lambda example: bool(example["question"] is not None and example["question"].strip()))
train_set = train_set.filter(lambda example: bool(example["answer"] is not None and example["answer"].strip()))
print(f"Size train after removing NaNs: {len(train_set)}\n")

print(f"Original size test: {len(test_set)}")
test_set = test_set.filter(lambda example: bool(example["context"] is not None and example["context"].strip()))
test_set = test_set.filter(lambda example: bool(example["question"] is not None and example["question"].strip()))
test_set = test_set.filter(lambda example: bool(example["answer"] is not None and example["answer"].strip()))
print(f"Size train after removing NaNs: {len(test_set)}")

We also build the full set consisting of both train and test cleaned samples.

In [ ]:
full_set = concatenate_datasets([train_set, test_set])

## Build questions and contexts corpuses
Every filtered row contains a distinct context document while there are a few repeated questions.

In [ ]:
print(f"Total rows (train + test): {len(full_set)}")
print(f"Unique context docs: {len(set(full_set['context']))}")
print(f"Unique questions: {len(set(full_set['question']))}")

doc_corpus = full_set["context"]  # list of strings
question_corpus = list(set(full_set["question"])) # list of strings (eliminate duplicates)

# 1) 🔎 Preliminary Analysis
We use pandas dataframes to simplify data manipulation.

In [ ]:
df_train = train_set.to_pandas()
df_test = test_set.to_pandas()
df_tot = pd.concat([df_train, df_test], ignore_index=True)

# len(df_train), len(df_test), len(df_tot)

Visualize first 5 samples.

In [ ]:
df_tot.head(5)

## Fields length correlation
Visualize the Pearson correlation coefficient between the lengths of the 3 fields in the dataset (context, question, answer).

In [ ]:
# Add length column for each field
# Use apply(lambda x: len(x.split())) for word count instead of characters
df_tot['question_len'] = df_tot['question'].apply(len)
df_tot['answer_len'] = df_tot['answer'].apply(len)
df_tot['context_len'] = df_tot['context'].apply(len)

In [ ]:
df_tot[['question_len', 'answer_len', 'context_len']].corr()

There is no significant correlation between the lengths of any pair of fields. The highest (yet weak) correlation is between the context length and the answer length.

## Documents length
**We begin by analysing the docs *corpus consisting of the context doc*s in the dataset.** Some statistics about the documents length.

In [ ]:
print("Statistics on the contexts: ")
print(df_tot["context_len"].describe())

Now let's visualize the distribution of the length of the documents

In [ ]:
plt.figure(figsize=(12, 5))
sns.histplot(df_tot["context_len"], bins=50, kde=True, color='steelblue')
plt.title("Distribution of lengths of context documents")
plt.xlabel("Length of context (in characters)")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()

## TF-IDF vectorization
Fit the vectorizer to the context documents corpus.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=0.8, min_df=3, stop_words='english', max_features=5000)
vectorizer.fit(doc_corpus)

Obtain vocabulary and print its size.

In [ ]:
vocab = vectorizer.get_feature_names_out()
print(len(vocab))

Build the tf-idf signatures of the context documents. Show the average number of features for each document.

In [ ]:
X = vectorizer.transform(doc_corpus)  # Matrix (12000, 5000) containing the docs tf-idf signatures
vocab_count_per_doc = (X != 0).sum(axis=1)  # Matrix (12000, 1) containing the non-zero entries in the docs signatures
vocab_count_per_doc = np.array(vocab_count_per_doc).flatten()  # Convert to 1D array

print("Avg features count per doc:", vocab_count_per_doc.mean()) # Different features contained in a doc on average

## Clustering w/ K-Means
** Cit riguardo al context su cui si basa analisi
We aim to cluster the context documents to identify patterns using the kmeans algorithm. We exploit the previously computed tf-idf vectorization of the documents.

We apply the elbow rule to identify an appropriate value for the number of clusters k. This consists of applying the algorithm for increasing values of k and choose the value after which improvements become marginal.

In [ ]:
from sklearn.cluster import KMeans

k_range = range(5, 60)  # Can be changed
inertia = []

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=SEED, max_iter=1000, n_init=10)
    # Requires X to be defined
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)

In [ ]:
# Plot for elbow method
plt.figure(figsize=(10, 6))
plt.plot(k_range, inertia, marker='o')
plt.xticks(k_range, rotation=45)
plt.grid(True)
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow method to determine optimal k')
plt.tight_layout()
plt.show()

The plot doesn't present an elbow. Inertia seems to keep decreasing without ever plateauing. We try using a different clustering quality metric to identify the optimal k, ie. the silhouette score.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

k_range = range(5, 60)  # Can be changed
scores = []

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=SEED, max_iter=1000, n_init=10)
    # Requires X to be defined
    labels = kmeans.fit_predict(X)
    score = silhouette_score(X, labels)
    scores.append(score)

In [ ]:
# Plot for elbow method
plt.figure(figsize=(12, 6))
plt.plot(k_range, scores)
plt.xlabel('Number of clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs. k')
plt.xticks(k_range, rotation=45)
plt.grid(True)
plt.show()

Although the plot still doesn't show a clear elbow, using this new metric we can observe that we get only marginal improvements for $k>50$. We therefore choose $k=50$ as the optimal value for the number of clusters.

In [ ]:
from sklearn.cluster import KMeans

k = 50  # Replace with your chosen number of clusters
kmeans = KMeans(n_clusters=k, random_state=SEED, max_iter=100000, n_init=30)
kmeans.fit(X)

# Assign cluster labels to each document
df_tot['cluster'] = kmeans.labels_

In [ ]:
from sklearn import metrics

print("Intrinsic evaluation measures")
print("-"*30)
print("Within-cluster sum-of-squares (Inertia):", kmeans.inertia_)
print("Silhouette coefficient:", metrics.silhouette_score(X, kmeans.labels_))

By plotting the most relevant tokens for clusters we observe that docs relating to similar topics are brought together.

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# PCA to reduce TF-IDF vectors for 2D visualization
pca = PCA(n_components=2)
scatter_plot_points = pca.fit_transform(X.toarray())

# Assume you've already clustered the data with KMeans
labels = kmeans.labels_

# Plot using a built-in colormap
x_axis = scatter_plot_points[:, 0]
y_axis = scatter_plot_points[:, 1]

plt.figure(figsize=(10, 8))
scatter = plt.scatter(x_axis, y_axis, c=labels, cmap='tab10')
plt.title("K-Means Clustering of Documents (Colored by Cluster)")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]

# Show the 10 most relevant tokens for a few clusters
for i in range(min(10, k)):
    print(f"Cluster {i}:")
    top_terms = [vocab[ind] for ind in order_centroids[i, :10]]
    print(", ".join(top_terms))
    print()

## Hierarchical Clustering
Given the difficulty of identifying a clear elbow (at least for reasonably low values of k) and the consequent difficulty in visualizing the docs clusters, we resort to hierarchical clustering using Ward's method as linkage method which aims at minimizing intra-cluster variance.

In [ ]:
from sklearn.decomposition import TruncatedSVD
from scipy.cluster.hierarchy import dendrogram, linkage
import scipy.spatial.distance as ssd

# For faster computation and plotting reduce this number (eg. 1000) to consider only a subset of docs
N_SAMPLES = X.shape[0]
X_subset = X[:N_SAMPLES]

# SVD reduction to make the tf-idf matrix more easily "tractable"
svd = TruncatedSVD(n_components=50, random_state=SEED)
X_reduced = svd.fit_transform(X_subset)

print(f"Reduced shape: {X_reduced.shape}")

# Compute pair-wise distances matrix
distance_matrix = ssd.pdist(X_reduced, metric='euclidean')

# Hierarchical clustering with 'ward' method
linkage_matrix = linkage(distance_matrix, method='ward')

In [ ]:
# Dendogram plot
plt.figure(figsize=(15, 7))
dendrogram(linkage_matrix, truncate_mode='level', p=8)
plt.title(f"Hierarchical Clustering Dendrogram (first {N_SAMPLES} documents)")
plt.xlabel('Document index')
plt.ylabel('Distance')
plt.tight_layout()
plt.show()

The dendogram allows us to identify 6 coarse macroclusters. We extract these macroclusters by cutting the dendogram horizontally such as to obtain at most 6 subtrees.

In [ ]:
from scipy.cluster.hierarchy import fcluster

# Cut the dendogram tree horizontally such as to form at most t flat macroclusters
cluster_labels = fcluster(linkage_matrix, t=6, criterion='maxclust')


We now visualize the most relevant tokens for each macrocluster.

In [ ]:
for cluster_id in range(1, 7):  # fcluster gives cluster labels starting from 1
    # Get indices of documents in this macrocluster
    doc_indices = np.where(cluster_labels == cluster_id)

    # Compute mean tf-idf scores for all terms in this cluster
    cluster_tfidf = X[doc_indices].mean(axis=0).A1

    # Get top 10 terms
    top_term_indices = cluster_tfidf.argsort()[::-1][:10]
    top_terms = vocab[top_term_indices]

    print(f"\nTop terms in macrocluster {cluster_id}:")
    print(", ".join(top_terms))

## Topic Modeling with LDA

In this section, we use **Latent Dirichlet Allocation (LDA)** to identify topics in our text data.
LDA assumes that each document is a mixture of topics and each topic is a distribution over words.
We will use `scikit-learn`'s implementation of LDA to extract latent topics from the corpus.



We begin by importing the LDA model and other utilities from `scikit-learn`.
We prepare a document-term matrix using `CountVectorizer` (with stop word removal and frequency thresholds) for LDA topic modeling.



In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# Use CountVectorizer instead of TF-IDF for LDA
count_vectorizer = CountVectorizer(max_df=0.8, min_df=5, stop_words='english', max_features=10000)
X_counts = count_vectorizer.fit_transform(doc_corpus)
terms = count_vectorizer.get_feature_names_out()

Set `n_topics`. Then initialize and fit an LDA model with custom alpha and beta priors to extract document-topic distributions from the count matrix.



In [ ]:
# Set the number of topics (eg. 10)
n_topics = 10

lda = LatentDirichletAllocation(
    n_components=n_topics,
    random_state=SEED,
    max_iter=100,
    doc_topic_prior=0.1,      # alpha
    topic_word_prior=0.01,    # beta
)

doc_topic_dist = lda.fit_transform(X_counts)

We extract the top terms for each topic by sorting the LDA component weights in descending order.


In [ ]:
# Extract top words for each topic
n_top_words = 10  # Set how many top words to extract
topics = []

for i, topic_weights in enumerate(lda.components_):
    top_terms_indices = topic_weights.argsort()[:-n_top_words - 1:-1]
    top_terms = [terms[i] for i in top_terms_indices]
    topics.append(top_terms)
    print(f"Topic {i}: {top_terms}")

We evaluate the LDA model using perplexity and log-likelihood to assess its fit to the data.


In [ ]:
# 🔍 Perplexity
perplexity = lda.perplexity(X_counts)
print(f"Model perplexity: {perplexity:.2f}")

# 📈 Log-likelihood
log_likelihood = lda.score(X_counts)
print(f"Model log-likelihood: {log_likelihood:.2f}")


We train an LDA model and compute a pseudo-cv coherence score by averaging word co-occurrences across top topic terms.


In [ ]:
from itertools import combinations

# Build binary term-document matrix
X_binary = (X_counts > 0).astype(int)

# Reverse index: word → index
term_index = {term: idx for idx, term in enumerate(terms)}

# Compute "pseudo-c_v" coherence
def topic_coherence(topic_words):
    indices = [term_index[word] for word in topic_words if word in term_index]
    if len(indices) < 2:
        return 0
    pair_scores = []
    for w1, w2 in combinations(indices, 2):
        docs_w1 = X_binary[:, w1]
        docs_w2 = X_binary[:, w2]
        co_occur = (docs_w1.multiply(docs_w2)).sum()
        pair_scores.append(co_occur)
    return np.mean(pair_scores) if pair_scores else 0

# Average coherence over all topics
coherences = [topic_coherence(t) for t in topics]
avg_coherence = np.mean(coherences)

print(f"Pseudo-cv coherence score (based on co-occurrence): {avg_coherence:.4f}")


### Compare results with different number of topics

We evaluate LDA models across different topic numbers using perplexity and log-likelihood, visualize the results, and print top words for the best model.


In [ ]:
# Define range of topics to test
k_values = range(5, 31, 5)
# Logging
perplexities = []
log_likelihoods = []
lda_models = []

print("Starting model evaluation...")
for k in k_values:
    lda = LatentDirichletAllocation(n_components=k, random_state=SEED, learning_method='batch')
    lda.fit(X_counts)

    perplexities.append(lda.perplexity(X_counts))
    log_likelihoods.append(lda.score(X_counts))
    lda_models.append(lda)

    print(f"k={k} → Perplexity: {perplexities[-1]:.2f}, LogLikelihood: {log_likelihoods[-1]:.2f}")

In [ ]:
# Plot results
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(k_values, perplexities, marker='o', color='blue')
plt.title("Perplexity vs Number of Topics")
plt.xlabel("Number of Topics")
plt.ylabel("Perplexity")
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(k_values, log_likelihoods, marker='o', color='green')
plt.title("Log-Likelihood vs Number of Topics")
plt.xlabel("Number of Topics")
plt.ylabel("Log-Likelihood")
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Select best model (with lowest perplexity)
best_index = np.argmin(perplexities)
best_k = k_values[best_index]
best_model = lda_models[best_index]

print(f"\nBest model: {best_k} topics (Perplexity: {perplexities[best_index]:.2f})")

n_top_words = 10
# Display top words for each topic
print("\nTop words for each topic:\n" + "-"*40)
for i, topic_weights in enumerate(best_model.components_):
    top_words_indices = topic_weights.argsort()[:-n_top_words-1:-1]
    top_words = [terms[idx] for idx in top_words_indices]
    print(f"Topic {i+1}: {', '.join(top_words)}")

We define a function to compute topic coherence by measuring log co-occurrence ratios of word pairs across documents.


In [ ]:
from itertools import combinations
from collections import defaultdict
import numpy as np

def compute_topic_coherence(topics, documents, top_n=10):
    """
    Computes a simple coherence metric for each topic.
    - topics: list of topics (each is a list of words)
    - documents: list of documents (already tokenized)
    """
    # Build inverted index: word -> set of document indices
    word_doc_freq = defaultdict(set)
    for i, doc in enumerate(documents):
        for word in set(doc):
            word_doc_freq[word].add(i)

    topic_coherences = []
    for topic in topics:
        topic = topic[:top_n]
        pair_scores = []
        for w1, w2 in combinations(topic, 2):
            docs_w1 = word_doc_freq.get(w1, set())
            docs_w2 = word_doc_freq.get(w2, set())
            intersection = docs_w1 & docs_w2
            # Add 1 for smoothing
            score = (len(intersection) + 1) / (min(len(docs_w1), len(docs_w2)) + 1)
            pair_scores.append(np.log(score))
        if pair_scores:
            topic_coherences.append(np.mean(pair_scores))
        else:
            topic_coherences.append(0)

    return topic_coherences, np.mean(topic_coherences)


We tokenize documents, extract top topic words, compute topic coherence scores, and print both individual and average values.


In [ ]:
tokenized_docs = [doc.lower().split() for doc in doc_corpus]

n_top_words = 10
topics = [[terms[i] for i in topic.argsort()[:-n_top_words-1:-1]] for topic in lda.components_]
topic_scores, avg_score = compute_topic_coherence(topics, tokenized_docs)

for i, score in enumerate(topic_scores):
    print(f"Topic {i+1}: Coherence = {score:.4f}")
print(f"\n📊 Average topic coherence: {avg_score:.4f}")

## Indexing for keyWord searching

In [ ]:
!pip install -q python-terrier==0.11.0

Now we prepare the training data for keyword indexing with PyTerrier by cleaning, reformatting, and assigning document IDs compatible with the retrieval engine.


In [ ]:
import pyterrier as pt
import pandas as pd

# Initialize PyTerrier
if not pt.started():
    pt.init()

# Use the training set
df = df_train.copy()

# Basic document preparation (no cleaning)
df = df.dropna(subset=["context"]).drop_duplicates(subset=["context"])
df = df[df["context"].str.strip().str.len() > 20]

# Rename columns for Terrier compatibility
df = df.reset_index(drop=True).reset_index()  # creates a numeric index
df = df.rename(columns={"index": "docno", "context": "text"})  # 'docno' and 'text' are expected



After indexing the documents with PyTerrier, we define a keyword-based search function using TF-IDF to retrieve and rank the most relevant contexts for a given query.


In [ ]:
df["docno"] = df["docno"].astype(str)


# Indexing the documents
indexer = pt.DFIndexer("./pt_index", overwrite=True)
index_ref = indexer.index(df["text"], df["docno"])
index = pt.IndexFactory.of(index_ref)

# Keyword Search Function

def search_keyword_pt(query, index, df, top_n=5):
    """
    Search for the top-N most relevant documents to a query using PyTerrier.
    """
    retriever = pt.BatchRetrieve(index, wmodel="TF_IDF")
    results = retriever.search(query).head(top_n)

    print(f"\nTop {top_n} documents most similar to the query '{query}':\n")
    for rank, row in enumerate(results.itertuples(), 1):
        doc_idx = int(row.docno)
        print(f"Rank {rank} - Document {doc_idx} - Score: {row.score:.4f}")
        print(df.loc[doc_idx, "text"][:300] + "...")
        print("-" * 80)

# Example Query

search_keyword_pt("artificial intelligence", index, df, top_n=3)

# 2) Advanced Analysis

## BERTopic modeling

In [ ]:
!pip install bertopic sentence-transformers umap-learn hdbscan

We train a BERTopic model on the context documents corpus, view topic info and top words, and visualize the results interactively.


In [ ]:
from bertopic import BERTopic

# Create and train the model
topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(doc_corpus)

# General info about topics
topic_info = topic_model.get_topic_info()
print(topic_info.head(10))  # first 10 topics

# Interactive visualization (only in Colab/Jupyter)
topic_model.visualize_topics()

# Show top words for each topic
for i in range(10):
    print(f"Topic {i}")
    print(topic_model.get_topic(i))
    print("-" * 40)


We compute the Silhouette Score for BERTopic by embedding non-outlier documents and measuring clustering quality.


In [ ]:
from sklearn.metrics import silhouette_score
from sentence_transformers import SentenceTransformer
import numpy as np

# 1. Remove documents assigned to topic -1 (outliers)
valid_indices = [i for i, t in enumerate(topics) if t != -1]
filtered_docs = [doc_corpus[i] for i in valid_indices]
filtered_labels = [topics[i] for i in valid_indices]

# 2. Create document embeddings (with the same model used by BERTopic)
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # same default as BERTopic
doc_embeddings = embedding_model.encode(filtered_docs, show_progress_bar=True)

# 3. Compute the Silhouette Score
score = silhouette_score(doc_embeddings, filtered_labels)
print(f"📊 Silhouette Score (BERTopic): {score:.4f}")



## Bidirectional LSTM question classifier
We aim to analyse the types of questions in the dataset. Since such questions are not labelled we first train a question classifier on the TREC dataset and use it to classify the RAG 12000 questions. The Text REtrieval Conference (TREC) Question Classification dataset contains 5500 labeled questions in training set and another 500 for test set.
Each question is labelled into one of 6 categories:
0. 'ABBR': Abbreviation.
1. 'ENTY': Entity.
2. 'DESC': Description and abstract concept.
3. 'HUM': Human being.
4. 'LOC': Location.
5. 'NUM': Numeric value.

In [ ]:
from datasets import load_dataset, concatenate_datasets

dataset_trec = load_dataset("trec", trust_remote_code=True)

print(dataset_trec)

train_set_trec = dataset_trec["train"] # Train split (5500 examples)
test_set_trec = dataset_trec["test"] # Test split (500 examples)

Clean the TREC dataset by removing examples where any field is empty or just whitespace from both train set and test set.

In [ ]:
print(f"Original size train: {len(train_set_trec)}")
train_set_trec = train_set_trec.filter(lambda example: bool(example["text"] is not None and example["text"].strip()))
train_set_trec = train_set_trec.filter(lambda example: bool(example["coarse_label"] is not None))
print(f"Size train after removing NaNs: {len(train_set_trec)}\n")

print(f"Original size test: {len(test_set_trec)}")
test_set_trec = test_set_trec.filter(lambda example: bool(example["text"] is not None and example["text"].strip()))
test_set_trec = test_set_trec.filter(lambda example: bool(example["coarse_label"] is not None))
print(f"Size test after removing NaNs: {len(test_set_trec)}\n")

full_set_trec = concatenate_datasets([train_set_trec, test_set_trec])

In [ ]:
train_x = train_set_trec["text"]
train_y = train_set_trec["coarse_label"]

test_x = test_set_trec["text"]
test_y = test_set_trec["coarse_label"]

Build training and validation Tensorflow datasets. Visualize the first training batch.

In [ ]:
import tensorflow as tf

train_tf = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_tf = tf.data.Dataset.from_tensor_slices((test_x, test_y))

train_dataset = train_tf.shuffle(buffer_size=10000).batch(batch_size=16).prefetch(tf.data.AUTOTUNE)
valid_dataset = valid_tf.batch(batch_size=16).prefetch(tf.data.AUTOTUNE)

for batch in train_dataset.take(1):
  print(batch)

Build the vectorizer.

In [ ]:
from tensorflow.keras.layers import TextVectorization

vectorizer = TextVectorization(max_tokens=10000)
# train_text = train_dataset.map(lambda text, label: text).unbatch()
vectorizer.adapt(train_x)

vocab = vectorizer.get_vocabulary()

Show vocabulary information and some sample question vectorizations.

In [ ]:
print(f"Vocab length: {len(vocab)}")
print(f"First 100 tokens: {vocab[:100]}")

for text in batch[0][:6].numpy():
    encoding = vectorizer([text]).numpy()[0]
    print('Question:     ', text.decode("utf-8"))
    print('Encoded:   ', encoding)
    print('Recovered: ',' '.join([vocab[i] for i in encoding]))
    print()

Define the network architecture:
- Bidirectional LSTM with embedding size of 64.

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
import tensorflow as tf

num_classes = train_set_trec.features["coarse_label"].num_classes

model = Sequential([
    tf.keras.Input(shape=(1,), dtype=tf.string),
    vectorizer,
    Embedding(input_dim=len(vectorizer.get_vocabulary()), output_dim=64, mask_zero=True),
    Bidirectional(LSTM(64)),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy']
)

model.summary()

Train the model.

In [ ]:
model.fit(train_dataset, epochs=10, validation_data=valid_dataset)

Evaluate the model.

In [ ]:
loss, accuracy = model.evaluate(valid_dataset)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

Plot confusion matrix.

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

trec_labels = ["ABBR", "ENTY", "DESC", "HUM", "LOC", "NUM"]

true_coarse = test_y
pred_coarse = np.argmax(model.predict(valid_dataset), axis=1)

cmd = ConfusionMatrixDisplay(confusion_matrix(true_coarse, pred_coarse, normalize='true'), display_labels=trec_labels)
cmd.plot()

We observe poor performance on ABBR type questions. We attribute this to the underrepresentation of such type of questions in both the training and testing set.

In [ ]:
from collections import Counter

# Count labels in trec train set
train_counts = Counter(example["coarse_label"] for example in train_set_trec)
test_counts = Counter(example["coarse_label"] for example in test_set_trec)

# Display counts for labels 0 to 5
for label in range(num_classes):
    print(f"Label {label} - Train: {train_counts[label]}, Test: {test_counts[label]}")

Finally we can apply the model to analyse the questions in the RAG 12000 dataset.

In [ ]:
questions = full_set["question"]

# Convert to a NumPy array or TensorFlow tensor
questions_tensor = tf.convert_to_tensor(questions, dtype=tf.string)

logits = model.predict(questions_tensor)
preds = np.argmax(logits, axis=1)
predicted_labels = [trec_labels[i] for i in preds]

# Add a column with the question type to the full df
df_tot["question_type"] = predicted_labels

Visualize an example for each question type.

In [ ]:
df_tot[["question", "question_type"]].groupby("question_type", group_keys=False).sample(n=1, random_state=SEED)

Plot question type distribution.

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter

# Count the labels
label_counts = Counter(predicted_labels)

# Sort labels by count in descending order
sorted_labels_counts = label_counts.most_common()
labels, counts = zip(*sorted_labels_counts)

# Plot
plt.figure(figsize=(8, 5))
plt.bar(labels, counts)
plt.xlabel("Label")
plt.ylabel("Count")
plt.title("Label Frequency (Sorted)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 3) Semantic Search

### Load sentence embedding model
We create an instance of a sentence embedding model based on BERT.

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

semb_model = SentenceTransformer('all-mpnet-base-v2')

Use it to pre-compute the context documents corpus embeddings.

In [ ]:
docs_embeddings = semb_model.encode(doc_corpus, convert_to_tensor=True, device='cuda')

Test retrieval of documents from the corpus given a query. Cosine similarity between query and document is used as distance metric.

In [ ]:
queries = [
    'A man is eating pasta.',
    'Someone in a gorilla costume is playing a set of drums.',
    'A cheetah chases prey on across a field.'
]

k = 1
# Set the maximum number of matches to retrieve
top_k = min(k, len(doc_corpus))

# Iterate over queries
for query in queries:
    # Embed query
    query_embedding = semb_model.encode(query, convert_to_tensor=True)
    # Use cosine-similarity and torch.topk to find the closest document in embedding space
    cos_scores = util.cos_sim(query_embedding, docs_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)
    # Print the result
    print(f"Query: \"{query}\"")
    print(f"Score: {score:.2f}")
    print("---------------------------------------")
    for score, idx in zip(top_results[0], top_results[1]):
        print(f"{doc_corpus[idx]}\n")
    print("\n\n")

## Load cross-encoder model
We create an instance of a cross-encoder model based on BERT.

In [ ]:
from sentence_transformers import SentenceTransformer, CrossEncoder, util

xenc_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

Test retrieval of documents from the corpus given a query. Using a cross-encoder generally leads to more accurate retrieval than the previous method based on cosine similarity alone. But it is also considerably slower since we cannot exploit the pre-computed documents embeddings.

In [ ]:
queries = [
    'A man is eating pasta.',
    'Someone in a gorilla costume is playing a set of drums.',
    'A cheetah chases prey on across a field.'
]

k = 1
# Set the maximum number of matches to retrieve
top_k = min(k, len(doc_corpus))

# Iterate over queries
for query in queries:
    # Prepare model inputs
    model_inputs = [[query, doc] for doc in doc_corpus]
    # Predict similarity score
    scores = xenc_model.predict(model_inputs)
    # Print the result
    print(f"Query: \"{query}\"")
    print("---------------------------------------")
    for idx in np.argsort(-scores)[0:top_k]:
        print(f"Score: {scores[idx]:.4f} - Document: \"{doc_corpus[idx]}\"")
    print("\n\n\n\n")

## Retrieve and rerank
In order to avoid having to compute the embedding of each $(query, document)$ pair we can use a hybrid approach.

We first perform fast retrival of the top $k$ most similar documents by cosine similarity.

In [ ]:
query = "What is the Berry Export Summary 2028 and what is its purpose?"

k = 100
top_k = min(k, len(doc_corpus))

# Embed query with sentence embdding modeel
query_embedding = semb_model.encode(query, convert_to_tensor=True)
# Use cosine-similarity and torch.topk to find the highest top_k similarity scores
cos_scores = util.cos_sim(query_embedding, docs_embeddings)[0]
top_results = torch.topk(cos_scores, k=top_k)
# print(top_results[0], top_results[1])

We then use the cross-encoder to rerank them by relevance wrt the query.

In [ ]:
doc_idxs = top_results[1].tolist()
# Consider only the documents identified in the previous step
model_inputs = [[query, doc_corpus[idx]] for idx in doc_idxs]

# Predict relevance scores with cross-encoder model
cross_scores = xenc_model.predict(model_inputs)

Finally we take the resulting most relevant documents (after reranking).

In [ ]:
print("Cross-encoder model re-ranking results")
print(f"Query: \"{query}\"")
print("---------------------------------------")
# Take top 3 of the reranked documents
for idx in np.argsort(-cross_scores)[:3]:
    print(f"Score: {cross_scores[idx]:.4f}\nDocument: \"{model_inputs[idx]}\"\n\n")

We create a class implementing such Retrieve&Rerank strategy for later use.

Even though our corpus only consists of 12000 documents the class will make use of a FAISS index. This way it can scale and be used as a retriever even for larger corpuses.

In [ ]:
!pip install faiss-cpu

In [ ]:
import faiss
import torch
import numpy as np
from sentence_transformers import util

class DocRetrieveRerank:
    def __init__(self, docs, semb_model, xenc_model, device='cpu'):
        self.docs = docs
        self.semb_model = semb_model
        self.xenc_model = xenc_model
        self.device = device

        self.n_docs = len(docs)
        # Precompute embeddings and convert to numpy
        self.docs_embeddings = semb_model.encode(docs, convert_to_numpy=True, normalize_embeddings=True, device=self.device)

        # Build FAISS index (inner product == cosine similarity on normalized vectors)
        self.index = faiss.IndexFlatIP(self.docs_embeddings.shape[1])
        self.index.add(self.docs_embeddings)

    """
    Retrieves and reranks documents for a given query.

    Args:
        query (str): The input query.
        k (int): Number of top documents to return.

    Returns:
        most_rel_docs (List[str]): Top-k reranked documents.
        most_rel_doc_idxs (List[int]): Indices in original doc list.
    """
    def retrieve(self, query, k=1):
        k_semb = min(50 * k, self.n_docs)
        k_xenc = min(k, self.n_docs)

        # Encode and normalize query embedding obtaining a (embedding_size) vector
        query_embedding = self.semb_model.encode(query, convert_to_numpy=True, normalize_embeddings=True)
        # Since FAISS expects a (num_queries, embedding_size) matrix
        query_embedding = query_embedding.reshape(1, -1)

        # Search top-k in FAISS
        cos_scores, doc_idxs = self.index.search(query_embedding, k_semb)
        # Since FAISS returns a (num_queries, k) matrix
        doc_idxs = doc_idxs[0]

        # Prepare cross-encoder inputs
        xenc_inputs = [[query, self.docs[idx]] for idx in doc_idxs]
        cross_scores = self.xenc_model.predict(xenc_inputs)

        # These idxs refer to xenc_inputs (not self.docs)
        most_rel_idxs = np.argsort(-cross_scores)[:k_xenc]
        most_rel_docs = [xenc_inputs[idx][1] for idx in most_rel_idxs]
        # Map the idxs to the original document corpus
        most_rel_doc_idxs = [doc_idxs[idx] for idx in most_rel_idxs]

        return most_rel_docs, most_rel_doc_idxs


# 4) BERT Models Fine-Tuning

The task we aim to solve here is understanding if the answer to a question is contained in a given context document. To solve this task we make use of both the RAG 12000 dataset and the RAG hallucination dataset. The latter contains $(question, context)$ pairs for which the answer cannot be answered from the context. We fine-tune a `BERT` based classifier to solve this task.




We load the RAG hallucination dataset.

In [ ]:
# Data loading
hallucination_dataset = load_dataset("neural-bridge/rag-hallucination-dataset-1000")

hall_train_set = hallucination_dataset["train"]
hall_test_set = hallucination_dataset["test"]

Clean the data by removing rows with empty entries.

In [ ]:
# Data cleaning
hall_train_set = hall_train_set.filter(lambda example: bool(example["context"] is not None and example["context"].strip()))
hall_train_set = hall_train_set.filter(lambda example: bool(example["question"] is not None and example["question"].strip()))
hall_train_set = hall_train_set.filter(lambda example: bool(example["answer"] is not None and example["answer"].strip()))

hall_test_set = hall_test_set.filter(lambda example: bool(example["context"] is not None and example["context"].strip()))
hall_test_set = hall_test_set.filter(lambda example: bool(example["question"] is not None and example["question"].strip()))
hall_test_set = hall_test_set.filter(lambda example: bool(example["answer"] is not None and example["answer"].strip()))

Convert it to dataframes. Introduce **answerable** column assigning label 1 to RAG 12000 answers and label 0 to hallucinated answers in the train and test sets.


In [ ]:
# Convert to pandas df
hall_df_train = hall_train_set.to_pandas()
hall_df_test = hall_test_set.to_pandas()

# RAG 12000
df_train['labels'] = 1
df_test['labels'] = 1

# RAG hallucination
hall_df_train['labels'] = 0
hall_df_test['labels'] = 0



We create balanced and shuffled train and evaluation datasets by combining real and hallucinated samples.

The RAG hallucination dataset only contains 800 samples in the train split so we have to sample an equivalent number of samples from the RAG 12000 dataset in order to get a balanced training set.



In [ ]:
# Create mixed dataframe by concatenation
train_dataset = pd.concat([df_train.sample(n=800, random_state=SEED), hall_df_train], ignore_index=True)
eval_dataset = pd.concat([df_test.sample(n=200, random_state=SEED), hall_df_test], ignore_index=True)

# Shuffle
train_dataset = train_dataset.sample(frac=1, random_state=SEED).reset_index(drop=True)
eval_dataset = eval_dataset.sample(frac=1, random_state=SEED).reset_index(drop=True)

We display a random sample of 20 training examples and check the total size of the training dataset.


In [ ]:
train_dataset.sample(n=10, random_state = SEED)

We convert the train and evaluation DataFrames to Hugging Face datasets.


In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_dataset)
eval_dataset = Dataset.from_pandas(eval_dataset)

We tokenize the context-question pairs with BERT, apply padding/truncation, and format the datasets for PyTorch training and evaluation.


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize(example):
    return tokenizer(
        example["context"],
        example["question"],
        padding="max_length",
        truncation=True,
        max_length=512
    )

train_dataset = train_dataset.map(tokenize, batched=True)
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

eval_dataset = eval_dataset.map(tokenize, batched=True)
eval_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [ ]:
!pip install transformers
!pip install torch torchvision

We load a BERT model for binary sequence classification using the `bert-base-uncased` checkpoint.


In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

After model training, this function enables evaluation by calculating accuracy, precision, recall, and F1 score based on predictions and labels.


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='weighted', zero_division=0
    )
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
!pip install transformers --upgrade
!pip install datasets --upgrade

To prepare for training, we configure key settings such as batch size, evaluation frequency, learning rate, and checkpointing strategy.


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./bert-finetune-results",
    num_train_epochs=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,

    eval_strategy="steps",   # Enable evaluation during training
    save_strategy="steps",         # Save models during training
    eval_steps=400,                # Evaluate every 400 steps (~half an epoch)
    save_steps=400,                # Save every 400 steps
    load_best_model_at_end=True,  # Restore best checkpoint (requires evaluation_strategy)
    metric_for_best_model="accuracy",

    save_total_limit=2,
    seed=SEED,
    fp16=True,                     # Use if GPU supports it (speeds up training)
    do_eval=True,
    do_train=True,
)

At this stage, we initialize the Trainer with the model, datasets, metrics, and early stopping, then start fine-tuning.


In [ ]:
from transformers import Trainer, EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,     # Define separately
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

trainer.train()

Once training is complete, we evaluate the model on the validation set and print the resulting metrics.


In [ ]:
results = trainer.evaluate()
print(results)

To perform inference, we move the model to the appropriate device and define a function that classifies a question-context pair as answerable or not.


In [ ]:
import torch

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to the device (do this once after loading the model)
model.to(device)

def classify_question(context, question):
    # Tokenize and move input tensors to the same device
    inputs = tokenizer(context, question, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Run model in inference mode (no gradients)
    with torch.no_grad():
        outputs = model(**inputs)
        prediction = outputs.logits.argmax(dim=1).item()

    return "Answerable" if prediction == 1 else "Unanswerable"

A set of labeled test cases is defined to evaluate the model’s ability to distinguish between answerable and unanswerable questions based on given contexts.


In [ ]:
test_cases = [
    # Answerable
    ("The Pacific Ocean is the largest and deepest of Earth's oceanic divisions.", "What is the largest ocean on Earth?", "Answerable"),
    ("Einstein developed the theory of relativity in the early 20th century.", "Who developed the theory of relativity?", "Answerable"),
    ("Mount Everest is the highest mountain in the world above sea level.", "What is the tallest mountain in the world?", "Answerable"),
        ("The Amazon rainforest is located in South America and is the largest tropical rainforest in the world.",
     "Where is the Amazon rainforest located?",
     "Answerable"),
    ("The Great Wall of China was built over several centuries to protect Chinese states against invasions.",
     "Why was the Great Wall of China built?",
     "Answerable"),
    ("The water cycle involves processes like evaporation, condensation, and precipitation.",
     "What are the main processes in the water cycle?",
     "Answerable"),
    ("Mars is the fourth planet from the Sun and is often called the Red Planet.",
     "Which planet is known as the Red Planet?",
     "Answerable"),
    ("Photosynthesis is the process by which green plants use sunlight to synthesize food from carbon dioxide and water.",
     "What is the process by which plants make food using sunlight?",
     "Answerable"),
    # Unanswerable
    ("The Pacific Ocean is the largest and deepest of Earth's oceanic divisions.", "What is the capital of Germany?", "Unanswerable"),
    ("The iPhone 13 was released in September 2021.", "What is the tallest building in Dubai?", "Unanswerable"),
    ("Shakespeare wrote many plays including Macbeth and Hamlet.", "How far is the moon from Earth?", "Unanswerable"),
    ("The human heart has four chambers: two atria and two ventricles.",
     "What is the capital of Canada?",
     "Unanswerable"),
    ("The Mona Lisa is a famous painting created by Leonardo da Vinci.",
     "When was the Eiffel Tower constructed?",
     "Unanswerable"),
    ("George Washington was the first President of the United States.",
     "What is the boiling point of mercury?",
     "Unanswerable"),
    ("Lightning is a sudden electrostatic discharge during a storm.",
     "Who is the CEO of Google?",
     "Unanswerable"),
    ("The Titanic sank in the North Atlantic Ocean in 1912 after hitting an iceberg.",
     "How many bones are in the human body?",
     "Unanswerable"),
]

To evaluate model performance, we generate predicted labels on the evaluation set, then visualize the results using a confusion matrix.


In [ ]:
eval_dataset.column_names  # View all columns

In [ ]:
# args (context, question)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

eval_zipped = zip(eval_dataset["context"], eval_dataset["question"], eval_dataset["labels"])


# Extract true labels
true_labels = ["Answerable" if label == 1 else "Unanswerable" for _, _, label in eval_zipped]
predicted_labels = []

eval_zipped = zip(eval_dataset["context"], eval_dataset["question"], eval_dataset["answerable"])
# Loop through each test case and print the result
for context, question, expected in eval_zipped:
    result = classify_question(context, question)
    #print(f"Context: {context}")
    #print(f"Question: {question}")
    #print(f"Expected Output: {expected}")
    #print(f"Model Output: {result}")
    #print("-" * 50)
    predicted_labels.append(result)

# Confusion matrix
cm = confusion_matrix(true_labels, predicted_labels, labels=["Answerable", "Unanswerable"])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Answerable", "Unanswerable"])
disp.plot()
plt.show()


# 5) LLM

In this section we load a quantized large language model (`Mistral-7B GPTQ`) and uses it to classify whether a question is answerable based on a given context. It:



1.   Loads the tokenizer and model (`AutoGPTQForCausalLM`) from HuggingFace.
2.   Builds prompts with optional reasoning examples (few-shot learning).
1.   Uses the model to generate answers and classifies the result as Answerable or Unanswerable.
1.   Evaluates performance on a set of test cases using zero-shot, one-shot, and few-shot strategies.
2.   Calculates and prints accuracy and a result table.






In [ ]:
!pip install ninja
!pip uninstall -y auto-gptq
!git clone https://github.com/PanQiWei/AutoGPTQ.git
%cd AutoGPTQ
!BUILD_CUDA_EXT=1 python setup.py install
!pip install gekko
!pip install -U datasets


!pip install -U transformers optimum auto-gptq

!pip uninstall -y optimum auto-gptq

!pip install git+https://github.com/huggingface/optimum.git@main
!pip install auto-gptq

!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118


In [ ]:
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM
import torch
import re

In [ ]:
model_name = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ"
device = "cuda:0" if torch.cuda.is_available() else "cpu"

print("🔄 Loading of tokenizer and model...")
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoGPTQForCausalLM.from_quantized(
    model_name,
    device=device,
    use_safetensors=True,
    trust_remote_code=True
)

In [ ]:
# ───── PROMPT BUILDER ─────
def build_prompt(context, question, examples=None):
    if examples is None:

        prompt = (
            "Instruction: Given a context and a question, determine whether the question "
            "is answerable based on the information in the context. "
            "Answer only with 'Answerable' or 'Unanswerable'.\n\n"
            f"Context: {context}\n"
            f"Question: {question}\n"
            f"Answerable:"
        )
    else:
        # ONE-SHOT / FEW-SHOT with Reasoning
        prompt = ""
        for ex in examples:
            prompt += (
                f"Context: {ex[0]}\n"
                f"Question: {ex[1]}\n"
                f"Reasoning: {ex[3]}\n"
                f"Answerable: {ex[2]}\n\n"
            )
        prompt += f"Context: {context}\nQuestion: {question}\nReasoning:"
    return prompt

# ───── CLASSIFIER WITH POST-PROCESSING ─────
def classify_answerable(model, tokenizer, context, question, examples=None):
    prompt = build_prompt(context, question, examples)
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    with torch.no_grad():
        output = model.generate(input_ids=input_ids, max_new_tokens=50)
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract relevant part after 'Answerable:'
    if "Answerable:" in decoded:
        response = decoded.split("Answerable:")[-1].strip().lower()
    else:
        response = decoded.lower()

    # Post-processing with regex
    match = re.search(r"\b(answerable|unanswerable|yes|no)\b", response)
    if match:
        token = match.group(1)
        if token in ["answerable", "yes"]:
            return "Answerable"
        elif token in ["unanswerable", "no"]:
            return "Unanswerable"
    return "Unknown"

In [ ]:
# ───── FEW-SHOT EXAMPLES ─────
few_shot_examples = [
    ("Mount Everest is the tallest mountain above sea level.", "What is the tallest mountain in the world?", "Answerable", "The context says Mount Everest is the tallest."),
    ("The human body has 206 bones.", "What is the capital of Spain?", "Unanswerable", "The context is about anatomy, not geography."),
    ("Photosynthesis occurs in plant cells using sunlight.", "How do plants make food?", "Answerable", "The context explains the process of photosynthesis."),
]

# ───── TEST SET ─────
test_cases = [
    ("The Pacific Ocean is the largest and deepest of Earth's oceanic divisions.", "What is the largest ocean on Earth?", "Answerable"),
    ("Einstein developed the theory of relativity in the early 20th century.", "Who developed the theory of relativity?", "Answerable"),
    ("Mount Everest is the highest mountain in the world above sea level.", "What is the tallest mountain in the world?", "Answerable"),
    ("The Amazon rainforest is located in South America and is the largest tropical rainforest in the world.", "Where is the Amazon rainforest located?", "Answerable"),
    ("The Great Wall of China was built over several centuries to protect Chinese states against invasions.", "Why was the Great Wall of China built?", "Answerable"),
    ("The water cycle involves processes like evaporation, condensation, and precipitation.", "What are the main processes in the water cycle?", "Answerable"),
    ("Mars is the fourth planet from the Sun and is often called the Red Planet.", "Which planet is known as the Red Planet?", "Answerable"),
    ("Photosynthesis is the process by which green plants use sunlight to synthesize food from carbon dioxide and water.", "What is the process by which plants make food using sunlight?", "Answerable"),
    # Unanswerable
    ("The Pacific Ocean is the largest and deepest of Earth's oceanic divisions.", "What is the capital of Germany?", "Unanswerable"),
    ("The iPhone 13 was released in September 2021.", "What is the tallest building in Dubai?", "Unanswerable"),
    ("Shakespeare wrote many plays including Macbeth and Hamlet.", "How far is the moon from Earth?", "Unanswerable"),
    ("The human heart has four chambers: two atria and two ventricles.", "What is the capital of Canada?", "Unanswerable"),
    ("The Mona Lisa is a famous painting created by Leonardo da Vinci.", "When was the Eiffel Tower constructed?", "Unanswerable"),
    ("George Washington was the first President of the United States.", "What is the boiling point of mercury?", "Unanswerable"),
    ("Lightning is a sudden electrostatic discharge during a storm.", "Who is the CEO of Google?", "Unanswerable"),
    ("The Titanic sank in the North Atlantic Ocean in 1912 after hitting an iceberg.", "How many bones are in the human body?", "Unanswerable"),
]

In [ ]:
# ───── EVALUATION ─────
results = []
for idx, (context, question, expected) in enumerate(test_cases):
    zs_pred = classify_answerable(model, tokenizer, context, question)
    os_pred = classify_answerable(model, tokenizer, context, question, examples=[few_shot_examples[0]])
    fs_pred = classify_answerable(model, tokenizer, context, question, examples=few_shot_examples)

    results.append({
        "Index": idx + 1,
        "Expected": expected.lower(),
        "Zero-shot": zs_pred.lower(),
        "One-shot": os_pred.lower(),
        "Few-shot": fs_pred.lower(),
    })

df = pd.DataFrame(results)

# ───── ACCURACY ─────
def compute_accuracy(col):
    return (df[col] == df["Expected"]).sum() / len(df)

print("\n📊 Accuracy per strategia:")
print(f"Zero-shot : {compute_accuracy('Zero-shot')*100:.2f}%")
print(f"One-shot  : {compute_accuracy('One-shot')*100:.2f}%")
print(f"Few-shot  : {compute_accuracy('Few-shot')*100:.2f}%")

# ───── RESULTS ─────
print("\n📋 Risultati dettagliati:\n")
print(df.to_markdown(index=False))


In [ ]:
from sklearn.metrics import classification_report

# Ground truth
y_true = df["Expected"]

# Evaluation function per strategy
def print_metrics(strategy_name, y_pred):
    print(f"\n Metrics per {strategy_name}:")
    report = classification_report(y_true, y_pred, labels=["answerable", "unanswerable"], zero_division=0)
    print(report)

# Computing of metrics
print_metrics("Zero-shot", df["Zero-shot"])
print_metrics("One-shot", df["One-shot"])
print_metrics("Few-shot", df["Few-shot"])

# 6) RAG based Q&A
We aim at building a question answering bot. We will generate both regular responses and RAG responses in order to compare them and see if the retrieval system actually enriches the response.

As generative model we use **Llama-3.2-1B-Instruct** which is a decoder-only instruction-tuned LLM.




In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import login
from google.colab import userdata

# Gated model: hf token is required
hf_token = userdata.get('luca_hf_token')
login(hf_token)

model_name = "meta-llama/Llama-3.2-1B-Instruct"

Load the model and the corresponding tokenizer.

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
# For open-end generation since it's a decoder-only model
tokenizer.pad_token = tokenizer.eos_token

We instantiate the document retriever for RAG. It implements the Retrieve&Rerank strategy seen in section 'Retrieve and rerank'.

In [ ]:
# Instantiate the document retriever for RAG
context_retriever = DocRetrieveRerank(docs=doc_corpus, semb_model=semb_model, xenc_model=xenc_model, device='cuda')

Setup the generation pipeline and arguments

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 250,
    "return_full_text": False,
    "temperature": 0.3,
    "do_sample": True,
}

Define some auxiliary functions to generate response and RAG response.

In [ ]:
def generate_response(system_prompt, user_prompt, pipe, generation_args):
  messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
  output = pipe(messages, **generation_args)
  return output[0]['generated_text']

In [ ]:
def get_response(query, pipe, generation_args):
    n_docs = 2  # Can be changed
    most_rel_docs, _ = context_retriever.retrieve(query, n_docs)

    # Build system and user prompts
    system_prompt = (
        "You are a friendly chatbot. You answer user questions"
    )
    user_prompt = (
        "Answer the user question:" + query + "\n\n"
    )

    response = generate_response(system_prompt, user_prompt, pipe, generation_args)
    return response

In [ ]:
def get_rag_response(query, context_retriever, pipe, generation_args):
    n_docs = 2  # Can be changed
    most_rel_docs, _ = context_retriever.retrieve(query, n_docs)

    # Build system and user prompts
    system_prompt = (
        "You are a friendly chatbot. You answer user questions based on the question and the context documents you are provided with"
    )
    user_prompt = (
        "Answer the user question:" + query + "\n\n" +
        "Here are some context documents that could be useful to answer:\n```\n" + '\n\n'.join(most_rel_docs) + "\n```\n"
    )

    response = generate_response(system_prompt, user_prompt, pipe, generation_args)
    return response

Start the Q&A bot. Ask questions and compare regular answer and RAG answer.

In [ ]:
print("📘 Q&ABot ready! Type 'exit' to quit.")

while True:
  user_query = input("🗿You: ")
  if user_query.lower() in ['exit', 'quit']:
      break

  bot_response = "🤖 Bot: " + get_response(user_query, pipe, generation_args)
  bot_rag_response = "🤖Bot+📖RAG: " + get_rag_response(user_query, context_retriever, pipe, generation_args)

  print(bot_response)
  print(bot_rag_response)
  print()


# 7) Text-to-speech model:

Before setting up the text-to-speech pipeline, we install all necessary Python and system dependencies for audio processing and waveform handling.


In [ ]:
!pip install numpy scipy librosa unidecode inflect
#!apt-get update
#!apt-get install -y libsndfile1

Now we load the pre-trained Tacotron2 text-to-speech model from NVIDIA's TorchHub, move it to the GPU, and set it to evaluation mode.


In [ ]:
import torch
tacotron2 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tacotron2', model_math='fp16')
tacotron2 = tacotron2.to('cuda')
tacotron2.eval()

Next, we load NVIDIA's WaveGlow vocoder from TorchHub, remove weight normalization for inference, move it to the GPU, and set it to evaluation mode.


In [ ]:
waveglow = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_waveglow', model_math='fp16')
waveglow = waveglow.remove_weightnorm(waveglow)
waveglow = waveglow.to('cuda')
waveglow.eval()

In [ ]:
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tts_utils')

In [ ]:
from IPython.display import Audio, display

We integrated the RAG-based question-answering chatbot with a text-to-speech system, allowing it to generate spoken responses in real-time.

In [ ]:

print("📘 Q&ABot ready! Type 'exit' to quit.")

while True:
    user_query = input("🗿You: ")
    if user_query.lower() in ['exit', 'quit']:
        break

    # Step 1: Get RAG response
    bot_text = get_rag_response(user_query, context_retriever, pipe, generation_args)
    print("🤖Bot+📖RAG:", bot_text)

    # Step 2: TTS inference
    sequences, lengths = utils.prepare_input_sequence([bot_text])
    with torch.no_grad():
        mel, _, _ = tacotron2.infer(sequences, lengths)
        audio = waveglow.infer(mel)

    audio_numpy = audio[0].data.cpu().numpy()
    rate = 22050

    # Step 3: Play the audio
    display(Audio(audio_numpy, rate=rate))


#8) Question Generation Using Our Dataset

We will train a model to generate questions based on a given context and answer from our dataset.


This code uses the `valhalla/t5-base-qg-hl` model, a T5 variant fine-tuned for question generation.  
It generates questions based on a given context and answer from the RAG-12000 dataset.


In [ ]:
#  1. Install and Import Required Libraries
!pip install -q transformers

from datasets import load_dataset
from transformers import pipeline
import pandas as pd

#  2. Load the RAG 12000 Dataset
dataset = load_dataset("neural-bridge/rag-dataset-12000", split="train")
print("Sample data format:")
print(dataset[0])

#  3. Prepare Input for Question Generation
# We'll use context + answer to generate a question

def format_input(example):
    context = example['context']
    answer = example['answer']
    return f"generate question: context: {context} answer: {answer}"

# Prepare a small subset for testing
subset = dataset.select(range(5))  # Change range for more examples
formatted_inputs = [format_input(example) for example in subset]

# 4. Load a Pretrained Question Generation Model
from transformers import pipeline
qg_pipeline = pipeline("text2text-generation", model="valhalla/t5-base-qg-hl")

#  5. Generate Questions
generated_questions = []
for inp in formatted_inputs:
    result = qg_pipeline(inp, max_length=64, truncation=True)[0]['generated_text']
    generated_questions.append(result)

#  6. Display Results
results_df = pd.DataFrame({
    "Context": [ex['context'] for ex in subset],
    "Answer": [ex['answer'] for ex in subset],
    "Generated Question": generated_questions
})

# Display in Colab
from IPython.display import display
display(results_df)



In [ ]:
#  Try a custom example
custom_context = "Polimi is the best school in Italy"
custom_answer = "Polimi"
custom_input = f"generate question: context: {custom_context} answer: {custom_answer}"

custom_result = qg_pipeline(custom_input, max_length=64, truncation=True)[0]['generated_text']
print("Custom input:")
print("→", custom_result)


In [ ]:
context = (
    "The Great Barrier Reef is located off the coast of Queensland, Australia. "
    "It is the world's largest coral reef system and can even be seen from space. "
    "The reef is home to a vast range of marine life and is a popular destination for divers."
)

answer = "Queensland, Australia"

input_text = f"generate question: context: {context} answer: {answer}"

# Run the model
result = qg_pipeline(input_text, max_length=64, truncation=True)[0]['generated_text']

print("Generated Question:", result)


This code fine-tunes the `t5-small` model for question generation using context and answer pairs.  
The model learns to generate questions on the RAG-12000 dataset and is saved for later use.


In [ ]:



# 2. Import libraries
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
#  3. Load RAG-12000 dataset
dataset = load_dataset("neural-bridge/rag-dataset-12000")
train_dataset = dataset["train"]
test_dataset = dataset["test"]

#  4. Format dataset: (context + answer) => question
def format_example(example):
    return {
        "input_text": f"generate question: context: {example['context']} answer: {example['answer']}",
        "target_text": example["question"]
    }

train_dataset = train_dataset.map(format_example, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(format_example, remove_columns=test_dataset.column_names)

# 5. Tokenize data
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

def tokenize(batch):
    # Make sure there are no None values or unexpected types
    inputs = [x if isinstance(x, str) else "" for x in batch["input_text"]]
    targets = [x if isinstance(x, str) else "" for x in batch["target_text"]]

    input_enc = tokenizer(inputs, truncation=True, padding="max_length", max_length=512)
    target_enc = tokenizer(targets, truncation=True, padding="max_length", max_length=64)

    input_enc["labels"] = target_enc["input_ids"]
    return input_enc



train_tokenized = train_dataset.map(tokenize, batched=True)
test_tokenized = test_dataset.map(tokenize, batched=True)

#  6. Define training arguments
training_args = TrainingArguments(
    output_dir="./simple_qg_model",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    logging_steps=100,
    save_strategy="no",  # doesn't save checkpoints to save time/space
    report_to="none"
)

#  7. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    tokenizer=tokenizer
)

#  8. Train
trainer.train()

#  9. Save final model
model.save_pretrained("./simple_qg_model")
tokenizer.save_pretrained("./simple_qg_model")


In [ ]:
from transformers import pipeline

# Load your trained model and tokenizer
qg_pipeline = pipeline(
    "text2text-generation",
    model="./simple_qg_model",
    tokenizer="./simple_qg_model"
)


In [ ]:
context = (
    "Rome is the capital of Italy."
)

answer = "Rome"

input_text = f"generate question: context: {context} answer: {answer}"

# Generate question
result = qg_pipeline(input_text, max_length=64, truncation=True)[0]["generated_text"]

print("Generated Question:", result)


This code fine-tunes the larger `t5-base` model to improve on the `t5-small` for question generation on the RAG-12000 dataset.  
It uses context and answer pairs as input to train the model to generate corresponding questions.


In [ ]:
#  2. Import libraries
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer

#  3. Load RAG-12000 dataset
dataset = load_dataset("neural-bridge/rag-dataset-12000")
train_dataset = dataset["train"]
test_dataset = dataset["test"]

# 4. Format data: (context + answer) → question
def format_example(example):
    return {
        "input_text": f"generate question: context: {example['context']} answer: {example['answer']}",
        "target_text": example["question"]
    }

train_dataset = train_dataset.map(format_example, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(format_example, remove_columns=test_dataset.column_names)

#  5. Load tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")

#  6. Tokenize
def tokenize(batch):
    inputs = [x if isinstance(x, str) else "" for x in batch["input_text"]]
    targets = [x if isinstance(x, str) else "" for x in batch["target_text"]]

    model_inputs = tokenizer(inputs, truncation=True, padding="max_length", max_length=512)
    labels = tokenizer(text_target=targets, truncation=True, padding="max_length", max_length=64)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_tokenized = train_dataset.map(tokenize, batched=True)
test_tokenized = test_dataset.map(tokenize, batched=True)

#  7. Define training arguments

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./qg_model_t5base",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    learning_rate=3e-4,
    weight_decay=0.01,
    logging_steps=100,
    save_steps=500,
    logging_dir="./logs",
    disable_tqdm=False,
    report_to="none"  # This tells Hugging Face not to report to W&B
)


#  8. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    tokenizer=tokenizer
)

#  9. Train the model
trainer.train()

#  10. Save it
model.save_pretrained("./qg_model_t5base")
tokenizer.save_pretrained("./qg_model_t5base")

In [ ]:
from transformers import pipeline

# Load your fine-tuned model
qg_pipeline = pipeline(
    "text2text-generation",
    model="./qg_model_t5base",
    tokenizer="./qg_model_t5base")



In [ ]:


#  Example 1: Custom input
context = (
    "The human brain has 86 billion neurons"
)
answer = "86 billion"

input_text = f"generate question: context: {context} answer: {answer}"

# Generate the question
output = qg_pipeline(input_text, max_length=64, truncation=True)[0]["generated_text"]
print("Generated Question:", output)

#9) Question Generation Using RAG-Instruct Dataset

We will now fine-tune a model to do the new task on the RAG-Instruct Dataset **Dataset 4**,using context extracted from documents and the provided answer.


This code fine-tunes `t5-small` on the RAG-Instruct dataset using documents and answers to generate questions.  
The dataset is split into train and test sets, and the trained model is saved
for later inference.


In [ ]:


#  2. Import libraries
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer

#  3. Load RAG-Instruct dataset

dataset = load_dataset("FreedomIntelligence/RAG-Instruct")["train"]

# Split into 90% train, 10% test
dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = dataset["train"]
test_dataset = dataset["test"]


#  4. Format dataset: (documents + answer) => question
def format_example(example):
    documents = example.get("documents", [])
    context = " ".join(documents).strip()
    answer = example.get("answer", "").strip()
    question = example.get("question", "").strip()
    return {
        "input_text": f"generate question: context: {context} answer: {answer}",
        "target_text": question
    }

train_dataset = train_dataset.map(format_example, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(format_example, remove_columns=test_dataset.column_names)

#  5. Tokenize data
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

def tokenize(batch):
    inputs = [x if isinstance(x, str) else "" for x in batch["input_text"]]
    targets = [x if isinstance(x, str) else "" for x in batch["target_text"]]

    input_enc = tokenizer(inputs, truncation=True, padding="max_length", max_length=512)
    target_enc = tokenizer(targets, truncation=True, padding="max_length", max_length=64)

    input_enc["labels"] = target_enc["input_ids"]
    return input_enc

train_tokenized = train_dataset.map(tokenize, batched=True)
test_tokenized = test_dataset.map(tokenize, batched=True)

#  6. Define training arguments
training_args = TrainingArguments(
    output_dir="./simple_qg_model_raginstruct",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    logging_steps=100,
    save_strategy="no",
    report_to="none"
)

#  7. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    tokenizer=tokenizer
)

#  8. Train the model
trainer.train()

#  9. Save final model
model.save_pretrained("./simple_qg_model_raginstruct")
tokenizer.save_pretrained("./simple_qg_model_raginstruct")


In [ ]:
from transformers import pipeline

# Load your fine-tuned model
qg_pipeline = pipeline(
    "text2text-generation",
    model="./simple_qg_model_raginstruct",
    tokenizer="./simple_qg_model_raginstruct"
)


In [ ]:
# Example context + answer
context = (
    "The Amazon rainforest is the largest tropical rainforest in the world, spanning over several countries "
    "in South America. It plays a critical role in regulating the global climate and hosts an incredible "
    "variety of species."
)
answer = "The Amazon rainforest"

# Format input just like during training
input_text = f"generate question: context: {context} answer: {answer}"
# Generate the question
output = qg_pipeline(input_text, max_length=64, truncation=True)[0]["generated_text"]

# Display the result
print("Generated Question:", output)
